In [1]:
# 自动加载已修改的python文件
%reload_ext autoreload
%autoreload 2

import os
import utils as utils
import tensorflow as tf  
from sklearn.model_selection import train_test_split 
import numpy as np
from sklearn.metrics import f1_score 
from sklearn.metrics import accuracy_score   
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score  
from sklearn.metrics import confusion_matrix
import random

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '1'       # 使用第二块GPU（从0开始）

pwd = "/notebooks/17_LJS/TbsData/"  # 当前路径 

### 读取样本

In [2]:
# 获取所有样本路径
def getSamplesDir():  
    image_junk = []  # 垃圾样本
    image_normal = []  # 正常样本
    image_abnormal = []  # 异常样本
    image_total = []  
    

    imageJunk_dir = pwd+"TBS3/Junk"
    imageNormal_dir = pwd+"TBS3/Negative"
    imageAbnormal_dir = pwd+"TBS3/Positive"
    # imageAscus_dir = pwd+"TBS3/ASCUS/Positive"

    # 获取所有样本名称
    listJunk = os.listdir(imageJunk_dir)
    listNormal = os.listdir(imageNormal_dir)
    listAbnormal = os.listdir(imageAbnormal_dir)
    # listAscus = os.listdir(imageAscus_dir)
    
    # 路径+名称
    listJunk = [os.path.join(imageJunk_dir,listJunk[i]) for i in range(len(listJunk))]
    listNormal = [os.path.join(imageNormal_dir,listNormal[i]) for i in range(len(listNormal))]
    listAbnormal = [os.path.join(imageAbnormal_dir,listAbnormal[i]) for i in range(len(listAbnormal))] 
    # listAscus = [os.path.join(imageAscus_dir,listAscus[i]) for i in range(len(listAscus))]
    
    # image_total = listJunk+listNormal+listAbnormal+listAscus
    # return image_total,len(listJunk),len(listNormal),len(listAbnormal)+len(listAscus)  
    image_total = listJunk+listNormal+listAbnormal
    return image_total,len(listJunk),len(listNormal),len(listAbnormal)  

# 读取原始测试集
# 获取所有样本路径
def getSamplesDir_test():  
    image_junk = []  # 垃圾样本
    image_normal = []  # 正常样本
    image_abnormal = []  # 异常样本
    image_total = []  

    imageJunk_dir = pwd+"TBS3/TestEnhance/Junk"
    imageNormal_dir = pwd+"TBS3/TestEnhance/Negative"
    imageAbnormal_dir = pwd+"TBS3/TestEnhance/Positive"

    # 获取所有样本名称
    listJunk = os.listdir(imageJunk_dir)
    listNormal = os.listdir(imageNormal_dir)
    listAbnormal = os.listdir(imageAbnormal_dir)
    
    # 路径+名称
    listJunk = [os.path.join(imageJunk_dir,listJunk[i]) for i in range(len(listJunk))]
    listNormal = [os.path.join(imageNormal_dir,listNormal[i]) for i in range(len(listNormal))]
    listAbnormal = [os.path.join(imageAbnormal_dir,listAbnormal[i]) for i in range(len(listAbnormal))] 
    
    image_total = listJunk+listNormal+listAbnormal
    return image_total,len(listJunk),len(listNormal),len(listAbnormal)  

### 修改，在训练样本的同时分批次的读取样本

#### 获取所有样本的路径

In [3]:
allSamplesDir,junkLen,negativeLen,positiveLen = getSamplesDir()  # 读取样本  
X_test,junkLen_test,negativeLen_test,positiveLen_test = getSamplesDir_test()  # 读取测试集

# 将两部分样本加在一块
allSamplesDir = allSamplesDir + X_test
junkLen = junkLen + junkLen_test
negativeLen = negativeLen + negativeLen_test
positiveLen = positiveLen + positiveLen_test

datas = np.array(allSamplesDir)
print ("样本总数据量为：")
print datas.shape

print "其它样本数为："
print junkLen
print "正常样本数为："
print negativeLen
print "异常样本数为："
print positiveLen

样本总数据量为：
(425122,)
其它样本数为：
111486
正常样本数为：
127228
异常样本数为：
186408


### 解决验证集、测试集被增强的问题

In [4]:
allSamplesDir_NoEnhance = [] # 未进行数据增强的原始数据

# 未进行数据增强的原始数据中各类数据的数目
negativeSize_noEnhance = 0  
positiveSize_noEnhance = 0
junkSize_noEnhance = 0

for i in range(len(allSamplesDir)):
    if allSamplesDir[i].find('0.jpg')>=0 and allSamplesDir[i].find('Negative')>=0:  # 未增强数据：正常样本类别
        negativeSize_noEnhance = negativeSize_noEnhance+1 
        allSamplesDir_NoEnhance.append(allSamplesDir[i])
    
    if allSamplesDir[i].find('0.jpg')>=0 and allSamplesDir[i].find('Positive')>=0:  # 未增强数据：异常样本类别
        positiveSize_noEnhance = positiveSize_noEnhance+1
        allSamplesDir_NoEnhance.append(allSamplesDir[i])
        
    if allSamplesDir[i].find('0.jpg')>=0 and allSamplesDir[i].find('Junk')>=0:  # 未增强数据：其它样本类别
        junkSize_noEnhance = junkSize_noEnhance+1
        allSamplesDir_NoEnhance.append(allSamplesDir[i])

print "其它样本数:"+str(junkSize_noEnhance)
print "正常样本数:"+str(negativeSize_noEnhance)
print "异常样本数:"+str(positiveSize_noEnhance)

print "未增强前_所有样本的个数:"+str(negativeSize_noEnhance+positiveSize_noEnhance+junkSize_noEnhance) 

其它样本数:55743
正常样本数:31807
异常样本数:23301
未增强前_所有样本的个数:110851


#### 为“未增强前数据”生成样本标签

In [5]:
def geneLabel(allSamplesDir_NoEnhance):
    y = []
    for i in range(len(allSamplesDir_NoEnhance)):
        value = allSamplesDir_NoEnhance[i]
        if value.find('Junk')>=0:
            y.append([1,0,0])
        elif value.find('Negative')>=0:
            y.append([0,1,0])
        elif value.find('Positive')>=0:
            y.append([0,0,1])
        else:
            print("geneLabel函数中出现了无法解决的bug！") 
    return y

y = geneLabel(allSamplesDir_NoEnhance)

#### 按照样本统计各样本所对应的图片个数

##### 提取一个图片路径中样本标签的函数

In [6]:
# 输入：/notebooks/17_LJS/TbsData/TBS3/Junk/EYC37815_C000076C_0.jpg
# 输出：EYC37815
def getSampleBelong(fullpath):
    num=0  # fullpath中/的个数
    for i in range(len(fullpath)):
        if fullpath[i]=='/':
            num=num+1
    # print "num:"+str(num)
    
    index = 0 # 统计遍历中遇到的/个数
    for i in range(len(fullpath)):
        if fullpath[i]=='/':
            index=index+1
            if index==num:
                break
    
    result = ''
    for i in range(i+1,len(fullpath)):
        if fullpath[i]!='_':
            result=result+fullpath[i]
        else:
            break
    # print "result:"+result
    return result 

In [7]:
class Info:  # 样本标签信息
    def __init__(self):
        self.sampleName = ''     # 样本标签名称
        self.sampleNum = 0     # 样本标签对应的图片个数 

##### 统计各样本所对应的图片个数

In [8]:
countResult = []  # 统计结果
for i in range(len(allSamplesDir_NoEnhance)):
    sample = getSampleBelong(allSamplesDir_NoEnhance[i])  # 某一个样本标签名称
    # print "sample:"+sample
    
    # 判断sample是否在countResult中
    num=0 # 用于计数
    for j in range(len(countResult)): 
        if countResult[j].sampleName==sample: 
            break
        num=num+1
    # print "len(countResult):"+str(len(countResult))
    # print "j:"+str(j)
    
    if num>=len(countResult):  # 表示sample不在countResult中
        info_tmp = Info()
        info_tmp.sampleName = sample
        info_tmp.sampleNum  =1
        countResult.append(info_tmp)
    else:
        countResult[j].sampleNum = countResult[j].sampleNum+1

##### 输出统计信息

In [9]:
for i in range(len(countResult)): 
    print countResult[i].sampleName+"\t\t"+str(countResult[i].sampleNum)

EYC37815		390
ESY51427		227
EES57040		813
EHG81734		439
C40655-ASCUS		3918
218327		353
223248		1085
EYC37951		923
EYC37760		691
B007852-ISIL		984
222488-ISIL		2939
C40740-ASCUS		5858
B007497-ISIL		1045
C41911-ASCUS		720
219352-ISIL		802
WHN806071		457
223115-ASC-H		601
222267-ISIL		79
EHG81280		451
EES57430		676
WQS805029		384
EHG81189		482
B007400-ISIL		1651
EHG81113		641
216762		168
EES109493		392
GL801153		400
CA2657-ASCUS		410
C40741-ASCUS		1615
222113-ISIL		538
ESY51306		655
GL801272		453
218710		395
GL801145		602
C40531-ASCUS		912
223744-ASC-H		322
ESY51668		779
CA0741-ASCUS		273
EES57484		561
EHG81624		380
C42441-ASCUS		525
C40592-ASCUS		861
ESY51750		871
EES57361		990
C44346		996
GL801175		585
EES57376		1029
223657		32
223648		36
C42404-ASCUS		731
B004562		112
B005801		290
EHG81910		480
B008053-LSIL-2		499
221960-ISIL		432
B003802		369
218326		21
223523		162
C25811		434
EYC37755		650
EYC37753		800
219073		103
218967-ISIL		152
CA2463-ASCUS		241
CA2564-ASCUS		102
ESY51300		334
B0

In [10]:
allPictureNum = sum([countResult[i].sampleNum for i in range(len(countResult))])
print "验证未进行图片增强前_图片的总数为："+str(allPictureNum)

验证未进行图片增强前_图片的总数为：110851


##### 将countResult打乱顺序，注意shuffle每次打乱的顺序都不一样，但此处使用的是train_test_split函数，可以指定种子点

In [11]:
def myShuffle(inputs,random_state=888):
    X_train,_,_,_ = train_test_split(inputs,[1]*len(inputs),test_size = 0,random_state =random_state)
    return X_train

# random.shuffle(countResult)
countResult = myShuffle(countResult,random_state=20)  # 使用固定的种子点

#### 划分“未增强前数据”，训练集：验证集:测试集
##### 测试集采用原始测试集等大，训练集：验证集=8:2

In [12]:
test_num = 14326
train_num = 0.8*(allPictureNum-test_num)
val_num = 0.2*(allPictureNum-test_num)

print "train_num:"+str(train_num)
print "val_num:"+str(val_num)
print "test_num:"+str(test_num)


def splitTrain_Test(countResult):
    X_train = []
    X_val = []
    X_test = []

    allPictureNum = sum([countResult[i].sampleNum for i in range(len(countResult))])  # 验证未进行图片增强前_图片的总数
    train = 0  # 用于计数训练集中图片的个数
    val = 0  # 用于计数验证集中图片的个数
    test = 0  # 用于计数测试集中图片的个数
    for i in range(len(countResult)):
        if train<=train_num:
            X_train.append(countResult[i])
            train = train+countResult[i].sampleNum
        elif val<=val_num:
            X_val.append(countResult[i])
            val = val+countResult[i].sampleNum
        else:
            X_test.append(countResult[i])
            test = test+countResult[i].sampleNum
    return X_train,X_val,X_test

X_train_sampleName,X_val_sampleName,X_test_sampleName = splitTrain_Test(countResult) # 划分训练集：测试集=8:2 

X_train_num = sum([X_train_sampleName[i].sampleNum for i in range(len(X_train_sampleName))])
X_val_num = sum([X_val_sampleName[i].sampleNum for i in range(len(X_val_sampleName))])
X_test_num = sum([X_test_sampleName[i].sampleNum for i in range(len(X_test_sampleName))])

print "\n数据集划分结果："
print "X_train_num:"+str(X_train_num)
print "X_val_num:"+str(X_val_num)
print "X_test_num:"+str(X_test_num)


print "\n训练集：验证集="+str(float(X_train_num)/X_val_num)

train_num:77220.0
val_num:19305.0
test_num:14326

数据集划分结果：
X_train_num:77406
X_val_num:19737
X_test_num:13708

训练集：验证集=3.92187262502


##### 根据样本标签名称，生成标签名称所对应的图片路径（未经过数据增强）

In [13]:
# 判断某个标签名称是否在某个Info列表中
def inInfo(infos,sample):
    num = 0
    for j in range(len(infos)):
        if infos[j].sampleName==sample:
            break
        num=num+1
    if num<len(infos):  # 表示sample在infos中
        return True
    else:
        return False

def getPictureRealPath(X_train_sampleName,X_val_sampleName,X_test_sampleName,allSamplesDir_NoEnhance):
    X_train = []
    X_val = []
    X_test = []
    
    for i in range(len(allSamplesDir_NoEnhance)):
        value = allSamplesDir_NoEnhance[i]  # 图片路径
        sample = getSampleBelong(allSamplesDir_NoEnhance[i])  # value所对应的样本标签名称 
        if inInfo(X_train_sampleName,sample)==True:
            X_train.append(value)
        elif inInfo(X_val_sampleName,sample)==True:
            X_val.append(value)
        elif inInfo(X_test_sampleName,sample)==True:
            X_test.append(value) 
        else:
            print("getPictureRealPath发生了无法解决的bug")
    return X_train,X_val,X_test

X_train,X_val,X_test = getPictureRealPath(X_train_sampleName,X_val_sampleName,X_test_sampleName,allSamplesDir_NoEnhance) 

##### 为训练集和验证集生成标签

In [14]:
y_train = geneLabel(X_train)
y_val = geneLabel(X_val)
y_test = geneLabel(X_test)

In [15]:
print "未增强前数据集总大小为："+str(len(X_train)+len(X_val))
print ("训练集大小：")
print len(X_train)
print len(y_train) 
print("验证集大小：")
print len(X_val)
print len(y_val)
print("测试集大小：")
print len(X_test)
print len(y_test)

未增强前数据集总大小为：97143
训练集大小：
77406
77406
验证集大小：
19737
19737
测试集大小：
13708
13708


#### 统计“未增强前_训练集中各类数据的数量”

In [16]:
def countSample(X_train):
    junk = 0
    negative = 0
    positive = 0 
    for i in range(len(X_train)):
        value = X_train[i]
        if value.find('Junk')>=0:
            junk = junk+1
        elif value.find('Negative')>=0:
            negative = negative+1
        else:
            positive = positive+1 

    return [junk,negative,positive]

nums = countSample(X_train)
print "垃圾样本："+str(nums[0])+"  阴性样本："+str(nums[1])+"  阳性样本："+str(nums[2]) 

垃圾样本：40951  阴性样本：20544  阳性样本：15911


#### 至此，得到“未增强前数据的验证集”，下面根据“未增强前数据的训练集”生成“数据增强之后的训练集”

#### 注意：此处对Junk增强为原始1倍；Negative增强为原始2倍；Positive增强为原始3倍

In [17]:
X_train_Enhance = []  # 表示进行数据增强后的训练集  

positive_num = 0  # 对未增强前的阳性类别进行计数
for i in range(len(X_train)):
    value = X_train[i]
    if value.find('Positive')>=0:  # 表示原始样本是“异常类别”
        # 由于对阳性进行增强之后，样本量由过大，故选择性的对阳性进行数据增强
        positive_num=positive_num+1 
        if positive_num<=12500:  # 表示只对X_train中125000个样本进行数据增强
            for j in range(3):
                newValue = value.replace('0.jpg',str(j)+'.jpg')
                X_train_Enhance.append(newValue)
        else:
            X_train_Enhance.append(value)
    elif value.find('Negative')>=0:  # 表示原始样本是“正常类别”
        newValue = value.replace('0.jpg',str(0)+'.jpg')
        X_train_Enhance.append(newValue)
        newValue = value.replace('0.jpg',str(1)+'.jpg')
        X_train_Enhance.append(newValue)       
        #         newValue = value.replace('0.jpg',str(2)+'.jpg')
        #         X_train_Enhance.append(newValue)       
        #         newValue = value.replace('0.jpg',str(5)+'.jpg')
        #         X_train_Enhance.append(newValue)
    elif value.find('Junk')>=0:  # 表示原始样本是“其它类别”
        for j in range(1):
            newValue = value.replace('0.jpg',str(j)+'.jpg')
            X_train_Enhance.append(newValue)
        
        # 以前Junk类别没有经过增强。
        # 对于“其它类别”，没有进行数据增强，原始样本和增强后的样本相同
        # X_train_Enhance.append(value)
    else:
        print "run中发生了无法结局的bug"

print ("训练集增强之后_大小为：")+str(len(X_train_Enhance))

# random.shuffle(X_train_Enhance)  # 对X_train_Enhance进行乱序排列
X_train_Enhance = myShuffle(X_train_Enhance,random_state=666)

训练集增强之后_大小为：122950


#### 根据“进行数据增强后的训练集”，生成对应的标签

In [18]:
X_train = X_train_Enhance
y_train = []
for i in range(len(X_train)):
    value = X_train[i]
    if value.find('Junk')>=0:  # 正常样本类别
        y_train.append([1,0,0])
    elif value.find('Negative')>=0:  # 异常样本类别
        y_train.append([0,1,0])
    else:  # 其它样本类别
        y_train.append([0,0,1])
print ("X_train大小：")+str(len(X_train))
print ("y_train大小：")+str(len(y_train))

X_train大小：122950
y_train大小：122950


### 函数定义：随机从列表中截取指定索引范围内，指定数量的数据

In [19]:
# 生成一定范围内指定数目的无重复数（mi：下限；ma：上限；num：数目）
def random_without_same(mi, ma, num):
    temp = range(mi, ma)  # 生成[mi,ma]之间的所有数
    # random.shuffle(temp)  # 对数据乱序排列
    temp = myShuffle(temp) 
    return temp[0:num]  # 返回数组前num个数字

def randomCropSample(mi, ma, num,X_train,y_train):
    # 生成“用于测试模型的训练集”，原始训练集太大，随机选取属于原始训练集一定数量（和验证集等大）的样本
    randIndex_inTrain = random_without_same(mi,ma,num)

    # 获取用于评估模型的训练集样本
    X_train_evalua = []
    y_train_evalua = []
    for i in range(len(randIndex_inTrain)):
        index = randIndex_inTrain[i]
        X_train_evalua.append(X_train[index])
        y_train_evalua.append(y_train[index])
    return X_train_evalua,y_train_evalua

In [20]:
nums = countSample(X_train)
print "垃圾样本："+str(nums[0])+"  阴性样本："+str(nums[1])+"  阳性样本："+str(nums[2])

垃圾样本：40951  阴性样本：41088  阳性样本：40911


### 模型评价函数

In [29]:
import time

# 调用接口计算“混淆矩阵”
def analyResult(y_test,predicts):
    
    resultsAnaly = confusion_matrix(y_test,predicts)
    print("混淆矩阵：")
    print(resultsAnaly) 

    accuracy = accuracy_score(y_test, predicts, normalize=True)
    precision_tmp = precision_score(y_test, predicts,average="macro")
    f1_score_tmp = f1_score(y_test, predicts,average="macro")
    recall_score_tmp = recall_score(y_test,predicts,average='macro')

    print("准确率："+str(accuracy))
    print("精确率："+str(precision_tmp))
    print("召回率："+str(recall_score_tmp))
    print("f1值： "+str(f1_score_tmp)) 
    return accuracy,precision_tmp,recall_score_tmp,f1_score_tmp

# 分析混淆矩阵
def analysisConfusionMatrix(ConfusionMatrix):
    result = ConfusionMatrix
    
    # 垃圾污染阳性
    two = float(sum(result[0][2:3]))/float(sum(result[0][:3])+1e-8)
    
    # 阴性污染阳性
    three = float(sum(result[1][2:3]))/float(sum(result[1][:3])+1e-8)
    
    # 阳性丢失
    one = float(sum(result[2][:2]))/float(sum(result[2][:3])+1e-8)
    
    print "阳性丢失："+str(round(one,5))+"  垃圾污染阳性："+str(round(two,5))+"  阴性污染阳性："+str(round(three,5))
    return one,two,three

### 定义F_beta_score

In [30]:
# 分析：对于“异常类别”，和查准率相比较，查全率更加重要（查全率高才能降低假阴性率），所以beta应该大于一。 
#       但是也不能一味地追求查全率，查准率也有一定的重要性，所以使用使用“异常类别”的f_beta_score系数作为模型评价指标。

def F_beta_score(confusionMatrix,beta):  # confusionMatrix二分类任务混淆矩阵,beta为异常类别f1score系数。
    tp = confusionMatrix[0][0]
    fn = confusionMatrix[0][1]
    fp = confusionMatrix[1][0]
    tn = confusionMatrix[1][1]
    
    # 计算“正常类别”的F_beta值
    # p_normal = tp/(tp+fp)
    # r_normal = tp/(tp+fn)
    # f_beta1 = (1+beta1*beta1)*p_normal*r_normal/((beta1*beta1*p_normal)+r_normal)
    
    # 计算“异常类别”的F_beta值
    p_ab = tn/(tn+fn+0.00000001)
    r_ab = tn/(tn+fp+0.00000001)
    f_beta2 = (1+beta*beta)*p_ab*r_ab/((beta*beta*p_ab)+r_ab+0.00000001)
    
    return f_beta2

# 合并三分类混淆矩阵中“正常类别”和“垃圾类别”，然后调用上述F_beta_score
def F_beta_score_MergeNormalAndJunk(confusionMatrix,beta):
    newConfusionMatrix = []
    tp = confusionMatrix[0][0]+confusionMatrix[0][1]+confusionMatrix[1][0]+confusionMatrix[1][1]
    fn = confusionMatrix[0][2]+confusionMatrix[1][2]
    fp = confusionMatrix[2][0]+confusionMatrix[2][1]
    tn = confusionMatrix[2][2]
    
    newConfusionMatrix_one = []
    newConfusionMatrix_one.append(tp)
    newConfusionMatrix_one.append(fn)
    
    newConfusionMatrix_two = []
    newConfusionMatrix_two.append(fp)
    newConfusionMatrix_two.append(tn)
    
    newConfusionMatrix.append(newConfusionMatrix_one)
    newConfusionMatrix.append(newConfusionMatrix_two)
    
    return F_beta_score(newConfusionMatrix,beta)

### 读取ASCUS类别样本

In [31]:
# 获取ASCUS类别样本
def getSamplesDir_Ascus():   
    imageAscus_dir = pwd+"TBS3/ASCUS/Positive"

    # 获取所有样本名称 
    listAscus = os.listdir(imageAscus_dir)
    
    # 路径+名称 
    listAscus = [os.path.join(imageAscus_dir,listAscus[i]) for i in range(len(listAscus))]  
    return listAscus


X_ascus = getSamplesDir_Ascus()
y_ascus = [[0,1,1]]*len(X_ascus)

print "ASCUS类别样本数据量为："
print len(X_ascus)
print len(y_ascus)

ASCUS类别样本数据量为：
35476
35476


### 定义模型训练函数

#### 生成训练集子集，用于分析模型

In [32]:
X_train_evalua,y_train_evalua=randomCropSample(0,len(X_train),len(X_val),X_train,y_train)

nums = countSample(X_train_evalua)
print "垃圾样本："+str(nums[0])+"  阴性样本："+str(nums[1])+"  阳性样本："+str(nums[2])

垃圾样本：6546  阴性样本：6621  阳性样本：6570


In [33]:
nums = countSample(X_val)
print "垃圾样本："+str(nums[0])+"  阴性样本："+str(nums[1])+"  阳性样本："+str(nums[2])

nums = countSample(X_test)
print "垃圾样本："+str(nums[0])+"  阴性样本："+str(nums[1])+"  阳性样本："+str(nums[2])

垃圾样本：9805  阴性样本：5802  阳性样本：4130
垃圾样本：4987  阴性样本：5461  阳性样本：3260


#### 定义模型网络结构

In [34]:
import Build
import tensorflow as tf  

def trainFun(start,batch_size=64,weight_decay=1e-8,lr=1e-4,epochs=80,firstPred=True,modelPath=pwd+'jsL/keras_model/',loadmodel=None,modelname=None,reDir='info'):
    # 首先情况重定向文件
    f=open(reDir+'/info.txt','w+')
    print >> f,"" 
    f.close() 
    #  每次运行都将图重新清空
    tf.reset_default_graph()  
    batch_size = batch_size   
     # 实例化模型
    model = Build.MobileNetV1(class_num=3)   
    # 构造正则化损失函数
    cost = model.cost  # 基本的交叉熵损失函数
    l2_loss = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])   # L2正则化 
    weight_decay = weight_decay  # 权重衰减系数
    if weight_decay!=None:
        cost = cost + l2_loss*weight_decay 

    # 表示训练多少个batch对应一epoch 
    epoch = int(len(X_train)/batch_size+1e-9)  
    
    # 配置tensorflow实用显存的方式
    config = tf.ConfigProto(allow_soft_placement=True)  # 自动选择可以运行的设备 
    config.gpu_options.allow_growth = True  # Gpu内存按需增长 
    # config.gpu_options.per_process_gpu_memory_fraction = 0.3 
    saver = tf.train.Saver(max_to_keep=1) # 模型保存器 
    
    # 使用SGD优化器，并且使用权重衰减
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(lr,global_step,10000,0.92,staircase=True)
    # train = tf.train.GradientDescentOptimizer(learning_rate).minimize(model.cost,global_step=global_step)
    train = tf.train.AdamOptimizer(learning_rate).minimize(model.cost,global_step=global_step) 
    # train = tf.train.MomentumOptimizer(learning_rate,momentum=0.9,use_nesterov=True).minimize(model.cost,global_step=global_step) 

    with tf.Session(config = config) as sess:    
        sess.run(tf.global_variables_initializer())  # 初始化变量  
        
        # 判断是否加载与训练模型
        if loadmodel!=None:
            saver = tf.train.import_meta_graph(loadmodel+modelname)
            saver.restore(sess,tf.train.latest_checkpoint(loadmodel))

        val_accuracy_list = []  # 绘制验证集准确率变化曲线
        val_f1_list = []  # 绘制验证集f1_score变化曲线
        train_accuracy_list = []  # 绘制训练集准确率变化曲线
        cost_list_train = []
        cost_list_val = []

        val_f_beta_score_max = -1 # 记录val_f1的最大值
        val_f1_max = -1 # 记录val_f1的最大值
        # 设置训练轮数(设置迭代400epoch)
        STEPS = int(epoch*epochs)  
        print "STEPS:"+str(STEPS)
        for i in range(start,STEPS+start):   # 表示从索引为start开始训练样本（主要是为了均衡二次训练时样本被训练的机会）
            
            utils.printRd("............................第"+str(i)+"个batch............................",reDir)  
            if i%(int(0.2*epoch))==0 and (i!=0 or firstPred==True):  # 判断在进行训练之前是否进行预测。 
                
                # 处理ASCUS类别数据
                ascus_predicts,time_ascus = utils.myPredicts(sess,model.prob,model.inputs,model.is_training,X_ascus,batch_size,reDir)
                ascus_evalua_realLabel = utils.getRealLabel(y_ascus)
                ascus_predicts_realLabel = utils.getRealLabel2(ascus_predicts,countSample(X_train)) 
                ascus_confusion_matrix = confusion_matrix(ascus_evalua_realLabel,ascus_predicts_realLabel) 
                print "ASCUS类别混淆矩阵为："
                print np.array(ascus_confusion_matrix)
                analyResult(ascus_evalua_realLabel,ascus_predicts_realLabel)
                print "阳性类别F1分数："+str(round(F_beta_score_MergeNormalAndJunk(ascus_confusion_matrix,1),5))
                analysisConfusionMatrix(ascus_confusion_matrix)
                
                
                # 处理测试集
                test_predicts,time_test = utils.myPredicts(sess,model.prob,model.inputs,model.is_training,X_test,batch_size,reDir)
                test_realLabel = utils.getRealLabel(y_test)
                test_predicts_realLabel = utils.getRealLabel2(test_predicts,countSample(X_train)) 
                test_confusion_matrix = confusion_matrix(test_realLabel,test_predicts_realLabel) 
                print "测试集混淆矩阵为："
                print np.array(test_confusion_matrix)
                analyResult(test_realLabel,test_predicts_realLabel)
                print "阳性类别F1分数："+str(round(F_beta_score_MergeNormalAndJunk(test_confusion_matrix,1),5))
                analysisConfusionMatrix(test_confusion_matrix)
                #                 # 将测试集中预测错误的样本写入到文件中
                #                 for i in range(len(test_realLabel)):
                #                     # if test_realLabel[i]!=test_predicts_realLabel[i]:
                #                     if test_realLabel[i]!=test_predicts_realLabel[i] and (test_realLabel[i]==2 or test_predicts_realLabel[i]==2):  

                #                         value = X_test[i]
                #                         if value.find('/notebooks/17_LJS/TbsData/TBS3/Test/Negative/')>=0:
                #                             value = value.replace('/notebooks/17_LJS/TbsData/TBS3/Test/Negative/','')
                #                         elif value.find('/notebooks/17_LJS/TbsData/TBS3/Test/Positive/')>=0:
                #                             value = value.replace('/notebooks/17_LJS/TbsData/TBS3/Test/Positive/','')
                #                         elif value.find('/notebooks/17_LJS/TbsData/TBS3/Test/Junk/')>=0:
                #                             value = value.replace('/notebooks/17_LJS/TbsData/TBS3/Test/Junk/','')
                #                         else:
                #                             print "发生了错误！"

                #                         utils.printRd(value,'info_test')
                #                 break
                
                
                # 处理验证集
                val_predicts,time_val = utils.myPredicts(sess,model.prob,model.inputs,model.is_training,X_val,batch_size,reDir)
                val_realLabel = utils.getRealLabel(y_val)
                val_predicts_realLabel = utils.getRealLabel2(val_predicts,countSample(X_train)) 
                val_confusion_matrix = confusion_matrix(val_realLabel,val_predicts_realLabel) 
                print "验证集混淆矩阵为："
                print np.array(val_confusion_matrix)
                analyResult(val_realLabel,val_predicts_realLabel)
                print "阳性类别F1分数："+str(round(F_beta_score_MergeNormalAndJunk(val_confusion_matrix,1),5))
                analysisConfusionMatrix(val_confusion_matrix)
                
                
                # 处理训练集
                train_predicts,time_train = utils.myPredicts(sess,model.prob,model.inputs,model.is_training,X_train,batch_size,reDir)
                train_evalua_realLabel = utils.getRealLabel(y_train)
                train_predicts_realLabel = utils.getRealLabel2(train_predicts,countSample(X_train)) 
                train_confusion_matrix = confusion_matrix(train_evalua_realLabel,train_predicts_realLabel) 
                print "训练集混淆矩阵为："
                print np.array(train_confusion_matrix)
                analyResult(train_evalua_realLabel,train_predicts_realLabel)
                print "阳性类别F1分数："+str(round(F_beta_score_MergeNormalAndJunk(train_confusion_matrix,1),5))
                analysisConfusionMatrix(train_confusion_matrix)
                
                time_total = time_ascus+time_train+time_test+time_val  # 预测总时长
                num_total = len(X_train)+len(X_val)+len(X_test)+len(X_ascus)  # 所预测的样本总量
                predict_efficiency = num_total/time_total  # 求单位时间内模型预测细胞的数量
                print "预测总时长为："+str(time_total)
                print "预测细胞总数量为："+str(num_total)
                print "模型单位时间内预测细胞数量为："+str(round(predict_efficiency,0))
                break;
                
                
                # 记录“训练集准确率”、“验证集准确率”、“验证集F1”  
                train_predicts,_ = utils.myPredicts(sess,model.prob,model.inputs,model.is_training,X_train_evalua,batch_size,reDir)  
                val_predicts,_ = utils.myPredicts(sess,model.prob,model.inputs,model.is_training,X_val,batch_size,reDir)
                
                # 求真实标签
                train_evalua_realLabel = utils.getRealLabel(y_train_evalua)
                train_predicts_realLabel = utils.getRealLabel2(train_predicts,countSample(X_train))
                # print "train_predicts_realLabel:"
                # print train_predicts_realLabel
                
                val_realLabel = utils.getRealLabel(y_val)
                val_predicts_realLabel = utils.getRealLabel2(val_predicts,countSample(X_train))

                # 计算“准确率”等 
                train_accuracy = accuracy_score(train_evalua_realLabel,train_predicts_realLabel, normalize=True)
                val_accuracy = accuracy_score(val_realLabel,val_predicts_realLabel, normalize=True)
                val_f1 = f1_score(val_realLabel,val_predicts_realLabel,average="macro") 
                train_confusion_matrix = confusion_matrix(train_evalua_realLabel,train_predicts_realLabel)
                val_confusion_matrix = confusion_matrix(val_realLabel,val_predicts_realLabel)
                
                # 修改模型评价指标为“异常类别”的f_beta_score
                val_f_beta_score = F_beta_score_MergeNormalAndJunk(val_confusion_matrix,beta=1)  # beta=2，表示“异常类别”查全率的重要性是查准率的4倍
                
                # 训练集交叉熵
                cross_entropy_train = sess.run(-tf.reduce_mean(y_train_evalua*tf.log(tf.clip_by_value(train_predicts,1e-10,1.0))))
                cross_entropy_val = sess.run(-tf.reduce_mean(y_val*tf.log(tf.clip_by_value(val_predicts,1e-10,1.0))))
                
                # 对各个参数只保留五位小数（后面的位四舍五入处理）
                train_accuracy = round(train_accuracy,5)
                val_accuracy = round(val_accuracy,5)
                cross_entropy_train = round(cross_entropy_train,5)
                cross_entropy_val = round(cross_entropy_val,5)
                val_f1 = round(val_f1,5) 
                val_f_beta_score = round(val_f_beta_score,5)

                utils.printRd("训练集混淆矩阵为：",reDir)
                utils.printRd(np.array(train_confusion_matrix),reDir)
                utils.printRd(analysisConfusionMatrix(train_confusion_matrix),reDir)
                utils.printRd("验证集混淆矩阵为：",reDir)
                utils.printRd(np.array(val_confusion_matrix),reDir)
                utils.printRd(analysisConfusionMatrix(val_confusion_matrix),reDir)

                utils.printRd("第"+str(i)+"次迭代："+"train_accuracy="+str(train_accuracy)+" val_accuracy="+str(val_accuracy)+" cost="+str(cross_entropy_train)+" val_f1:"+str(val_f1)+ " val_f_beta_score:"+str(val_f_beta_score),reDir) 
 
                train_accuracy_list.append(train_accuracy)
                val_accuracy_list.append(val_accuracy)
                val_f1_list.append(val_f1) 
                cost_list_train.append(cross_entropy_train)
                cost_list_val.append(cross_entropy_val)
                utils.showPerformCurve(val_accuracy_list,val_f1_list,train_accuracy_list,reDir+'/perform.jpg') # 显示曲线 
                utils.showPerformCurve_cost(cost_list_train,cost_list_val,reDir+'/cost.jpg') # 显示曲线 
                
                if val_f1>val_f1_max:
                    val_f1_max = val_f1
                    utils.printRd("val_f1最大值更新："+str(val_f1_max),reDir)

                # 修改模型保存条件为：每次epoch判断一次val_f1是否大于val_f1_max,假如满足条件就保存一次模型
                if val_f_beta_score>val_f_beta_score_max:  # 这样做也就相当于earlystoping。
                    # vgg.save_npy(sess,pwd+'jsL/keras_model/vggPara-save_train_l2正则化解决过拟合.npy')     
                    # 因为模型中使用了BN层，参数无法提取，所以替换使用Saver.save函数保存模型
                    saver.save(sess, modelPath)  # 保存模型  
                    val_f_beta_score_max = val_f_beta_score
                    utils.printRd("model saved!",reDir)
                else:
                    utils.printRd("model not saved!",reDir)  

            # 每次选取batch_size个样本进行训练
            # 梯度下降训练模型参数
            start = (i*batch_size)%len(X_train)
            end = min(start+batch_size,len(X_train)) 
            xxx = utils.getMiniBatch4Train(X_train[start:end])
            sess.run(train, feed_dict={model.inputs: xxx, model.labels: y_train[start:end], model.is_training: True})  
        sess.close() 

### 训练模型

In [35]:
import tensorflow as tf
initial_depthwise = tf.truncated_normal([4, 4, 4, 4], 0.0, 0.001)

In [36]:
loadmodel = '/notebooks/17_LJS/model2019/Pangbo/'
modelname = 'model_new.meta'
reDir = 'info_test' 

modelPath='/notebooks/17_LJS/model2019/Pangbo/model_new'
trainFun(0,batch_size=256,weight_decay=1e-6,lr=1e-3,epochs=15,firstPred=True,modelPath=modelPath,loadmodel=loadmodel,modelname=modelname,reDir=reDir)

标记：
Tensor("Placeholder:0", shape=(?, 128, 128, 3), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 3), dtype=float32)
Tensor("Placeholder_2:0", dtype=bool)
start-shape:(?, 128, 128, 3)
the 1th stage-shape：(?, 64, 64, 32)
the 2th stage-shape：(?, 32, 32, 32)
the 3th stage-shape：(?, 16, 16, 64)
the 4th stage-shape：(?, 8, 8, 64)
Tensor("prob:0", shape=(?, 3), dtype=float32)
the 5th stage-shape：(?, 3)
INFO:tensorflow:Restoring parameters from /notebooks/17_LJS/model2019/Pangbo/model_new
STEPS:7200
ASCUS类别混淆矩阵为：
[[    0     0     0]
 [    0     0     0]
 [  232 10704 24540]]
混淆矩阵：
[[    0     0     0]
 [    0     0     0]
 [  232 10704 24540]]
准确率：0.6917352576389671
精确率：0.3333333333333333
召回率：0.23057841921298905
f1值： 0.27259397494001597
阳性类别F1分数：0.81778
阳性丢失：0.30826  垃圾污染阳性：0.0  阴性污染阳性：0.0
测试集混淆矩阵为：
[[4135  810   42]
 [  55 5160  246]
 [  10  154 3096]]
混淆矩阵：
[[4135  810   42]
 [  55 5160  246]
 [  10  154 3096]]
准确率：0.9039247154946017
精确率：0.9140013237623617
召回率：0.907910315463588
f1值： 0.

In [ ]:
%%javascript 
Jupyter.notebook.session.restart() 

In [ ]:
# x  = 0.01
# for i in range(15):
#     print "x:"+str(x)
#     x = x*0.92